# Using model to determine nutritional content

In [19]:
import pandas as pd
import numpy as np
import spacy

import sys
sys.path.append("/Users/maxkirwan/Desktop/Uni/Data Science MSc/Data Science Project/nutrition-insta")
import functions

import requests

In [36]:
posts = pd.read_csv("posts_with_nutritional_info.csv")
posts = posts.iloc[:,1:26]
posts['ingredients'] = posts['ingredients'].apply(lambda x: x[1:-1].split(','))

In [20]:
def get_usda_info_json(ingredient):
    
    headers = {}
    params = {'api_key': 'XBZBkznaG3lFCfED92u489nCZFlm9zdBNdH6lCEZ'}
    
    json_data = {
        'query': ingredient,
        'dataType': [
            'Foundation',
            'SR Legacy'
        ],
        'pageSize' : 1,
        'pageNumber' : 1,
        'sortBy': 'dataType.keyword',
        'sortOrder': 'asc',
    }

    response = requests.post('https://api.nal.usda.gov/fdc/v1/foods/list', params=params, headers=headers, json=json_data)
    
    return response.json()

In [44]:
def calc_nutrient_levels_recipe(ing_list):
    '''
    USDA food lookup for each ingredient in recipe.
    Then totals levels of required nutrients.
    To give total nutrient levels for whole recipe.
    '''
    
    ENERGY = 0     # KCAL
    PRO = 0        # G
    CHO = 0        # G
    FAT = 0        # G
    
    
    for ing in ing_list:
        
        # Query ingredient against FoodData Central database
        response = get_usda_info_json(str(ing))

        
        # If query returned a response
        if response:
        
            # Get list of nutrients
            nutrients = response[0]['foodNutrients']

            for nutrient in nutrients:

                if nutrient['name'] == 'Total lipid (fat)':
                    FAT += nutrient['amount']
                    print("added",ing,'//',nutrient['name'],nutrient['amount'])

                elif nutrient['name'] == 'Carbohydrate, by difference':
                    CHO += nutrient['amount']
                    print("added",ing,'//',nutrient['name'],nutrient['amount'])

                elif nutrient['name'] == 'Protein':
                    PRO += nutrient['amount']
                    print("added",ing,'//',nutrient['name'],nutrient['amount'])

                elif (nutrient['name'] == 'Energy' and nutrient['unitName'] == 'KCAL') or nutrient['name'] == 'Energy (Atwater General Factors)':
                    ENERGY += nutrient['amount']
                    print("added",ing,'//',nutrient['name'],nutrient['amount'])
                
                
    nutrient_levels = {'ENERGY': ENERGY,
                       'PRO' : PRO,
                       'CHO' : CHO,
                       'FAT' : FAT
                      }
    
    return nutrient_levels



def convert_dict_to_series_values(nutrient_levels_dict):
    
    return pd.Series(nutrient_levels_dict.values())

In [45]:
calc_nutrient_levels_recipe(posts['ingredients'].iloc[0])

added oats // Total lipid (fat) 6.31
added oats // Carbohydrate, by difference 69.9
added oats // Protein 13.2
added oats // Energy (Atwater General Factors) 389
added  milk // Protein 7.81
added  milk // Total lipid (fat) 11.0
added  milk // Carbohydrate, by difference 6.86
added  milk // Energy 157
added  water // Total lipid (fat) 0.94
added  water // Energy 90.0
added  water // Protein 19.0
added  water // Carbohydrate, by difference 0.08
added  sugar // Protein 0.0
added  sugar // Total lipid (fat) 0.32
added  sugar // Energy 385
added  sugar // Carbohydrate, by difference 99.6
added  peanut butter // Total lipid (fat) 49.4
added  peanut butter // Protein 24.0
added  peanut butter // Carbohydrate, by difference 22.7
added  peanut butter // Energy (Atwater General Factors) 632
added  banana // Total lipid (fat) 0.22
added  banana // Protein 0.73
added  banana // Carbohydrate, by difference 20.1
added  banana // Energy 85.0


{'ENERGY': 1738.0, 'PRO': 64.74, 'CHO': 219.23999999999998, 'FAT': 68.19}

In [49]:
# Adding cols to df for energy, prot, cho, fat

nutrient_cols = ['ENERGY','PROT','CHO','FAT']

posts[nutrient_cols] = posts['ingredients'].apply(calc_nutrient_levels_recipe).apply(convert_dict_to_series_values)

added oats // Total lipid (fat) 6.31
added oats // Carbohydrate, by difference 69.9
added oats // Protein 13.2
added oats // Energy (Atwater General Factors) 389
added  milk // Protein 7.81
added  milk // Total lipid (fat) 11.0
added  milk // Carbohydrate, by difference 6.86
added  milk // Energy 157
added  water // Total lipid (fat) 0.94
added  water // Energy 90.0
added  water // Protein 19.0
added  water // Carbohydrate, by difference 0.08
added  sugar // Protein 0.0
added  sugar // Total lipid (fat) 0.32
added  sugar // Energy 385
added  sugar // Carbohydrate, by difference 99.6
added  peanut butter // Total lipid (fat) 49.4
added  peanut butter // Protein 24.0
added  peanut butter // Carbohydrate, by difference 22.7
added  peanut butter // Energy (Atwater General Factors) 632
added  banana // Total lipid (fat) 0.22
added  banana // Protein 0.73
added  banana // Carbohydrate, by difference 20.1
added  banana // Energy 85.0
added pesto // Protein 4.16
added pesto // Total lipid (fat

added  parmesan // Total lipid (fat) 28.0
added  parmesan // Carbohydrate, by difference 12.4
added  parmesan // Energy 421
added  parmesan // Protein 29.6
added  cheese // Carbohydrate, by difference 2.44
added  cheese // Energy 408
added  cheese // Protein 23.3
added  cheese // Total lipid (fat) 34.0
added  garlic powder // Total lipid (fat) 0.38
added  garlic powder // Protein 6.62
added  garlic powder // Carbohydrate, by difference 28.2
added  garlic powder // Energy 143
added  black pepper // Total lipid (fat) 0.106
added  black pepper // Protein 0.715
added  black pepper // Carbohydrate, by difference 4.78
added  black pepper // Energy (Atwater General Factors) 22.9
added oats // Total lipid (fat) 6.31
added oats // Carbohydrate, by difference 69.9
added oats // Protein 13.2
added oats // Energy (Atwater General Factors) 389
added  chia seeds // Total lipid (fat) 56.1
added  chia seeds // Carbohydrate, by difference 17.1
added  chia seeds // Energy 612
added  chia seeds // Protei

added oats // Total lipid (fat) 6.31
added oats // Carbohydrate, by difference 69.9
added oats // Protein 13.2
added oats // Energy (Atwater General Factors) 389
added  vanilla // Total lipid (fat) 0.06
added  vanilla // Carbohydrate, by difference 12.6
added  vanilla // Energy 288
added  vanilla // Protein 0.06
added  cinnamon // Total lipid (fat) 5.29
added  cinnamon // Carbohydrate, by difference 44.4
added  cinnamon // Energy 253
added  cinnamon // Protein 7.05
added  chia seeds // Total lipid (fat) 56.1
added  chia seeds // Carbohydrate, by difference 17.1
added  chia seeds // Energy 612
added  chia seeds // Protein 21.0
added  vanilla // Total lipid (fat) 0.06
added  vanilla // Carbohydrate, by difference 12.6
added  vanilla // Energy 288
added  vanilla // Protein 0.06
added  vanilla // Total lipid (fat) 0.06
added  vanilla // Carbohydrate, by difference 12.6
added  vanilla // Energy 288
added  vanilla // Protein 0.06
added lemon juice // Protein 0.55
added lemon juice // Total l

added  milk // Protein 7.81
added  milk // Total lipid (fat) 11.0
added  milk // Carbohydrate, by difference 6.86
added  milk // Energy 157
added  vanilla // Total lipid (fat) 0.06
added  vanilla // Carbohydrate, by difference 12.6
added  vanilla // Energy 288
added  vanilla // Protein 0.06
added  vanilla // Total lipid (fat) 0.06
added  vanilla // Carbohydrate, by difference 12.6
added  vanilla // Energy 288
added  vanilla // Protein 0.06
added  seed // Total lipid (fat) 56.1
added  seed // Carbohydrate, by difference 17.1
added  seed // Energy 612
added  seed // Protein 21.0
added  cinnamon // Total lipid (fat) 5.29
added  cinnamon // Carbohydrate, by difference 44.4
added  cinnamon // Energy 253
added  cinnamon // Protein 7.05
added  garlic powder // Total lipid (fat) 0.38
added  garlic powder // Protein 6.62
added  garlic powder // Carbohydrate, by difference 28.2
added  garlic powder // Energy 143
added  coriander // Total lipid (fat) 17.8
added  coriander // Carbohydrate, by diff

In [50]:
posts

,postUrl,profileUrl,username,fullName,commentCount,likeCount,pubDate,description,location,imgUrl,...,language,score,description_preprocessed,includes_ingredient_list,ingredient_list,ingredients,ENERGY,PROT,CHO,FAT
0,https://www.instagram.com/p/Cgd_iR_vT2d/,https://www.instagram.com/all.about.oats,all.about.oats,Anushka Lodhi,0,2,2022-07-26T08:23:14.000Z,Chocolate fudge protein oatmeal🤎 💪\nIngredient...,"Ghaziabad, India",https://scontent-lhr8-2.cdninstagram.com/v/t51...,...,en,0.999996,chocolate fudge protein oatmeal \ningredients:...,True,"[{'ingredient': oats, 'quantity': 1/2, 'measur...","[oats, milk, water, sugar, peanut butter, ...",1738.0,64.740,219.240,68.190
1,https://www.instagram.com/p/Cgd8bLXDaCq/,https://www.instagram.com/a_m_eats,a_m_eats,Alice & Meg 🍴,3,12,2022-07-26T07:56:03.000Z,⁣Caprese Chicken with Pesto 🌿🍅⠀\n⠀\nThis dish ...,"Glasgow, United Kingdom",https://scontent-lhr8-1.cdninstagram.com/v/t51...,...,en,0.999997,⁣caprese chicken with pesto ⠀\n⠀\nthis dish is...,True,"[{'ingredient': pesto, 'quantity': 2, 'measure...",[pesto],372.0,4.160,6.930,36.400
2,https://www.instagram.com/p/CgdylqtOy9s/,https://www.instagram.com/hescottwellness,hescottwellness,"Natasha Hescott, RDN, CDN",0,1,2022-07-26T06:30:06.000Z,Looking for a fast recipe to make for lunch? T...,NaN,https://scontent-lhr8-1.cdninstagram.com/v/t51...,...,en,0.999996,looking for a fast recipe to make for lunch? t...,True,"[{'ingredient': ginger, 'quantity': 1, 'measur...","[ginger, tamari, hemp seeds, water, oil]",842.0,52.320,40.550,57.890
3,https://www.instagram.com/p/CgdyENyLgmv/,https://www.instagram.com/rainbowpiatto,rainbowpiatto,Rainbow Piatto,1,14,2022-07-26T06:25:32.000Z,Sweet & Savoury Mushroom Scones~🍄 This easy an...,Singapore / Singapura / 新加坡 / சிங்கப்பூர்,https://scontent-lhr8-1.cdninstagram.com/v/t51...,...,en,0.999998,sweet & savoury mushroom scones~ this easy and...,True,"[{'ingredient': mushrooms, 'quantity': 200, 'm...","[mushrooms, sugar, baking powder, salt, bu...",2039.8,63.910,200.320,192.559
4,https://www.instagram.com/p/CgdpvTNvrWt/,https://www.instagram.com/foodiesfood_court,foodiesfood_court,Foodies Food Court,43,1819,2022-07-26T05:12:47.000Z,Paneer Tikka Recipe😍\nSave it to try later♥️\n...,Bihar,https://scontent-lhr8-2.cdninstagram.com/v/t51...,...,en,0.999996,paneer tikka recipe\nsave it to try later\n\ni...,True,"[{'ingredient': flour, 'quantity': 1/2, 'measu...","[flour, carom seeds, ginger, red mirch, cu...",1896.9,67.205,253.400,167.906
5,https://www.instagram.com/p/CgdmRS0DWm4/,https://www.instagram.com/corner_to_discover,corner_to_discover,Shivangi Gupta,4,102,2022-07-26T05:04:54.000Z,MASALA PANEER 🥰\n\nTo set the paneer why to us...,NaN,NaN,...,en,0.999996,masala paneer \n\nto set the paneer why to use...,True,"[{'ingredient': milk, 'quantity': 1, 'measurem...","[milk, oregano, chili flakes, jeera powder]",1021.0,29.450,238.660,16.210
6,https://www.instagram.com/p/Cgdl5Uehh6u/,https://www.instagram.com/mixin._.goodness,mixin._.goodness,Sunaina,2,47,2022-07-26T04:39:12.000Z,Paneer Aaloo mix Paratha 👩‍🍳😃😋\n\nIngredients:...,NaN,https://scontent-lhr8-1.cdninstagram.com/v/t51...,...,en,0.999995,paneer aaloo mix paratha \n\ningredients: pane...,True,[],[],645.0,20.800,21.200,53.000
7,https://www.instagram.com/p/CgdgJ2eA8Tl/,https://www.instagram.com/sri_vani_cooking_cha...,sri_vani_cooking_channel,Srivani's Kitchen,1,9,2022-07-26T03:49:32.000Z,"Healthy chutney powder to serve with Idli, dos...",NaN,NaN,...,en,0.999997,"healthy chutney powder to serve with idli, dos...",True,"[{'ingredient': flax seeds, 'quantity': 1/2, '...","[flax seeds, coriander, urad, sesame, red ...",1712.0,58.290,101.530,139.780
8,https://www.instagram.com/p/CgdfLOSrAUG/,https://www.instagram.com/enzo_mapua,enzo_mapua,enzo mapua,3,48,2022-07-26T03:40:28.000Z,Boscaiola! @rumourhasit9 had cooked this for m...,NaN,https://scontent-lhr8-2.cdninstagram.com/v/t51...,...,en,0.999997,boscaiola! had cooked this for my family and i...,True,"[{'ingredient'

In [55]:
calc_nutrient_levels_recipe(posts['ingredients'].iloc[4])

added flour // Total lipid (fat) 1.52
added flour // Protein 11.4
added flour // Carbohydrate, by difference 74.4
added flour // Energy (Atwater General Factors) 357
added  carom seeds // Total lipid (fat) 56.1
added  carom seeds // Carbohydrate, by difference 17.1
added  carom seeds // Energy 612
added  carom seeds // Protein 21.0
added  ginger // Total lipid (fat) 0.75
added  ginger // Protein 1.82
added  ginger // Carbohydrate, by difference 17.8
added  ginger // Energy 80.0
added  red mirch // Protein 0.94
added  red mirch // Total lipid (fat) 0.1
added  red mirch // Carbohydrate, by difference 9.93
added  red mirch // Energy 44.0
added  cumin powder // Protein 17.8
added  cumin powder // Total lipid (fat) 22.3
added  cumin powder // Carbohydrate, by difference 44.2
added  cumin powder // Energy 375
added  black pepper powder // Total lipid (fat) 0.106
added  black pepper powder // Protein 0.715
added  black pepper powder // Carbohydrate, by difference 4.78
added  black pepper powd

{'ENERGY': 1896.9, 'PRO': 67.205, 'CHO': 253.4, 'FAT': 167.906}

In [54]:
posts['PROT']*4 + posts['CHO']*4 + posts['FAT']*9 - posts['ENERGY']

0       11.630
1       -0.040
2       50.490
3      750.151
4      896.674
5      197.330
6        0.000
7      185.300
8       16.519
9       57.040
10      15.274
11    -196.550
12       5.880
13      33.080
14       3.380
15    1523.580
16       7.833
17     826.173
18       0.000
19    1491.822
20       0.000
21      -0.400
22      87.438
23    -664.560
24     790.570
25      95.830
26     500.490
27       5.360
28    -483.150
29      50.490
30     263.710
31       0.000
32     102.670
33     508.320
34    -427.060
35     263.710
36     263.710
37     517.060
38       0.000
39       0.000
40      14.470
41      16.450
42      32.560
43      29.600
44     517.060
45     499.810
46     117.304
47    -236.820
48      -2.880
49       9.446
dtype: float64

In [56]:
posts.to_csv("posts_with_macro_nutrients.csv")